In [8]:
import os
import sys

import torch
from torch import nn
from tinynn.converter import TFLiteConverter

from config import build_model_config
from modelx.detectors import build_model

class DotDict(dict):
    def __init__(self, *args, **kwargs):
        super(DotDict, self).__init__(*args, **kwargs)

    def __getattr__(self, key):
        value = self[key]
        if isinstance(value, dict):
            value = DotDict(value)
        return value

def main_worker():
    device = torch.device('cpu')

    args = {'model':'yolov2', 'num_classes':20, 'conf_thresh':0.1, 'nms_thresh':0.5,
            'topk':100,
            'weight':'./weights/voc/yolov2/yolov2_epoch_51_75.34.pth'}
    args = DotDict(args) 

    # config
    model_cfg = build_model_config(args)

    # build model
    model = build_model(
        args=args, 
        model_cfg=model_cfg,
        device=device, 
        num_classes=args.num_classes,
        trainable=False
        )
    
    model.eval()
    
    dummy_input = torch.rand((1, 3, 640, 640))

    output_path = os.path.join("./", 'weights/onnx/', 'yolov2_epoch_51_75.34.tflite')

    # When converting quantized models, please ensure the quantization backend is set.
    torch.backends.quantized.engine = 'qnnpack'

    # The code section below is used to convert the model to the TFLite format
    # If you want perform dynamic quantization on the float models,
    # you may refer to `dynamic.py`, which is in the same folder.
    # As for static quantization (e.g. quantization-aware training and post-training quantization),
    # please refer to the code examples in the `examples/quantization` folder.
    converter = TFLiteConverter(model, dummy_input, output_path)
    converter.convert()


if __name__ == '__main__':
    main_worker()

Model: YOLOV2 ...
Build YOLOV2 ...
Model Configuration: 
 {'trans_type': 'ssd', 'multi_scale': [0.5, 1.5], 'backbone': 'darknet19', 'pretrained': True, 'stride': 32, 'neck': 'sppf', 'expand_ratio': 0.5, 'pooling_size': 5, 'neck_act': 'lrelu', 'neck_norm': 'BN', 'neck_depthwise': False, 'head': 'decoupled_head', 'head_act': 'lrelu', 'head_norm': 'BN', 'num_cls_head': 2, 'num_reg_head': 2, 'head_depthwise': False, 'anchor_size': [[17, 25], [55, 75], [92, 206], [202, 21], [289, 311]], 'iou_thresh': 0.5, 'loss_obj_weight': 1.0, 'loss_cls_weight': 1.0, 'loss_box_weight': 5.0, 'no_aug_epoch': -1, 'optimizer': 'sgd', 'momentum': 0.937, 'weight_decay': 0.0005, 'clip_grad': 10, 'ema_decay': 0.9999, 'ema_tau': 2000, 'scheduler': 'linear', 'lr0': 0.01, 'lrf': 0.01, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1}
Neck: sppf
Head: Decoupled Head


WARNING (tinynn.converter.operators.torch.prim) fmp_h.1 is of type int64, which is unsupported in TFLite, trying to downcast to int32
WARNING (tinynn.converter.operators.torch.prim) fmp_w.1 is of type int64, which is unsupported in TFLite, trying to downcast to int32
ERROR (tinynn.converter.base) Unsupported ops: aten::sort


Exception: Cannot continue due to fatal error